In [3]:
#05 统计bayes的每个样本reads数，写成A256G的形式，过滤，只过滤测序深度,不过滤频率。
#统计每个样本的突变数量
#dp>100并且在至少2个样本中出现过的才算。
#变异类型是snp或者del或ins，这里也筛选了。
#最终输出csv文件。

import csv
import os
import vcf
wenjianjia_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/yjq_sra_bioproject_rsv')
for i in wenjianjia_list:
    dirs = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/10change_lofreq/01filtered_csv_deep_sample/{i}'
    if not os.path.exists(dirs):
        os.makedirs(dirs)

for p in os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/group/10change_lofreq/vcf_pair'):
    prj_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/10change_lofreq/vcf_pair/{p}'
    
    bayes_filenames=os.listdir(prj_path)
    type_list= []
    dict_mut_num = {}#在几个样本中出现过。形成列表，在列表中的才算真正的突变。
    #找出突变在几个样本中出现
    for i in bayes_filenames:
        if i.endswith('vcf'):
            i_1 = i.split('.')[0]
            reads_pos_dict = {}
            path1 = prj_path+'/'+i
            vcf_reader = vcf.Reader(filename=path1)
            for record in vcf_reader:
    #             print(record)
    #             print(record.INFO)

                alt_count = record.INFO['AO']
                ref_count = record.INFO['RO']
                ref = str(record).split(',')[2].split('=')[1]
                type0 = record.INFO['TYPE']
                position = str(record).split(',')[1].split('=')[1]
                dp = record.INFO['DP']
                if type0 not in type_list:
                    type_list.append(type0)
                for i in range(0,len(record.INFO['TYPE'])):
                    alt1 = str(record).split('=')[4].replace(')','').replace('[','').replace(']','').replace(' ','')
                    alt2 = alt1.split(",")#转成列表,碱基
                    type2 = record.INFO['TYPE'][i]
                    info = f'{ref}_{position}_{alt2[i]}'
                    alt_count1 = alt_count[i]
                    af = record.INFO['AO'][i]/(record.INFO['AO'][i] + record.INFO['RO'])
    #                 print(af)
                    if dp > 100:
    #                     reads_pos_dict[info] = {'alt':alt_count1,'ref':ref_count,'type':type2,'allele_frequency':af}
                        if info not in dict_mut_num:
                            dict_mut_num[info] = 1
                        else:
                            dict_mut_num[info] += 1
    #         print(reads_pos_dict)



    filtered_list = []# 测序深度>100且出现在至少2个样本中的突变。
    for k,v in dict_mut_num.items():
        if v > 1:
            filtered_list.append(k)
    # print(filtered_list)


    #根据filtered_list筛选突变，输出csv。
    for i in bayes_filenames:
        if i.endswith('vcf'):
            i_1 = i.split('.')[0]
            reads_pos_dict = {}
            path1 = prj_path +'/'+i
            vcf_reader = vcf.Reader(filename=path1)
            for record in vcf_reader:
    #             print(record)
    #             print(record.INFO)

                alt_count = record.INFO['AO']
                ref_count = record.INFO['RO']
                ref = str(record).split(',')[2].split('=')[1]
                type0 = record.INFO['TYPE']
                position = str(record).split(',')[1].split('=')[1]
                dp = record.INFO['DP']
                if type0 not in type_list:
                    type_list.append(type0)
                for i in range(0,len(record.INFO['TYPE'])):
                    alt1 = str(record).split('=')[4].replace(')','').replace('[','').replace(']','').replace(' ','')
                    alt2 = alt1.split(",")#转成列表,碱基
                    type2 = record.INFO['TYPE'][i]
                    info = f'{ref}_{position}_{alt2[i]}'
                    alt_count1 = alt_count[i]
                    af = record.INFO['AO'][i]/(record.INFO['AO'][i] + record.INFO['RO'])    
                    if dp > 100:#即使有列表了也还是需要测序深度>100的这个条件，不然有可能是出现在样本多但是测序深度不行的。
                        if info in filtered_list:
                            if type2 == 'snp' or type2 == 'del' or type2 == 'ins':#筛选变异类型
                                reads_pos_dict[info] = {'alt':alt_count1,'ref':ref_count,'type':type2,'allele_frequency':af,'sample_exist':dict_mut_num[info]}
    #         print(reads_pos_dict)

            with open(f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/10change_lofreq/01filtered_csv_deep_sample/{p}/{i_1}.csv','w') as f:#表格可以加个频率
                writer = csv.writer(f)
                writer.writerow(["mutation","alt_reads","ref_reads","type","app","allele_frequency","sample_exist'"])
                for k,v in reads_pos_dict.items():
                    writer.writerow([k,v['alt'],v['ref'],v['type'],'freebayes',v['allele_frequency'],v['sample_exist']])
                    
                    
                    
                    

KeyError: 'AO'

In [2]:
#vcf_all里面有乱七八糟的单端数据啥的，只要pair的，把metadata手动变成csv(用wps)，筛选出来这些vcf，移入vcf_pair,这个是分过组的。
#/mnt/alamo01/users/yangjingqi/RSV_sra/group/09change_ref/00script/04vcf_all_to_pair/04vcf_all_to_pair.py
import os
import csv
#新建分组的文件夹

wenjianjia_list = os.listdir('/mnt/alamo01/users/yangjingqi/RSV_sra/yjq_sra_bioproject_rsv')
for i in wenjianjia_list:
    dirs = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/10change_lofreq/vcf_pair/{i}'
    if not os.path.exists(dirs):
        os.makedirs(dirs)
        
list_sra_pair=[]        
with open('/mnt/alamo01/users/yangjingqi/RSV_sra/group/pair_metadata.csv') as csvfile:
    csv_reader = csv.reader(csvfile)  
    for row in csv_reader:
        if row[0] != 'Run':
            list_sra_pair.append(row[0])
        
for i in wenjianjia_list:
    vcf_prj_path = f'/mnt/alamo01/users/yangjingqi/RSV_sra/group/10change_lofreq/vcf_result/{i}'
    vcf_list = os.listdir(vcf_prj_path)
    for n in vcf_list:
        n1 = n.split('.')[0]
        if n1 in list_sra_pair:
            os.system(f'cp /mnt/alamo01/users/yangjingqi/RSV_sra/group/10change_lofreq/vcf_result/{i}/{n} /mnt/alamo01/users/yangjingqi/RSV_sra/group/10change_lofreq/vcf_pair/{i}/{n}')
    